In [16]:
import numpy as np 
import pandas as pd 
import xgboost as xg 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error as MSE 
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression

In [17]:
# Load the data 
dataset = pd.read_csv("validator_data.csv") 
X = dataset.drop(['effectiveness'], axis=1)
y = dataset['effectiveness']

In [18]:
X.head()

,id,total_validators,p_active_slots,p_created_blocks,non_empty_block,empty_block,vanilla_blocks,a_active_epochs,a_attested_epochs,a_attested_source,...,inc_distance,sync_rate,block_reward,validation_reward,total_reward,proposal_effectiveness,correctness,attestation_rate,att_effectiveness,partition_date
0,Sigma Prime,10001,83,83,82.0,0.0,51.0,2250225,2249558,2248718,...,1.021263,97.957517,6.568270,26.959453,33.527723,0.987952,0.992777,0.999704,0.971819,2024-01-17
1,Simply Staking,9428,88,88,88.0,0.0,0.0,2121300,2117898,2117410,...,1.038131,98.022129,11.981733,25.534410,37.516143,1.000000,0.988021,0.998396,0.950203,2024-01-17
2,Chorus One,8853,77,77,77.0,0.0,0.0,1991925,1988552,1987913,...,1.039940,97.778459,7.538779,23.694192,31.232970,1.000000,0.991658,0.998307,0.951957,2024-01-17
3,CryptoManufaktur,10001,80,80,80.0,0.0,0.0,2250225,2249817,2249157,...,1.019378,99.339419,12.021850,27.143702,39.165552,1.000000,0.995019,0.999819,0.975927,2024-01-17
4,Attestant (BVI) Limited,10001,69,69,69.0,0.0,8.0,2250225,2250199,2249164,...,1.018523,98.925253,6.736894,26.254547,32.991440,1.000000,0.995301,0.999988,0.977189,2024-01-17


In [19]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5388 entries, 0 to 5387
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      5388 non-null   object 
 1   total_validators        5388 non-null   int64  
 2   p_active_slots          5388 non-null   int64  
 3   p_created_blocks        5388 non-null   int64  
 4   non_empty_block         5329 non-null   float64
 5   empty_block             5329 non-null   float64
 6   vanilla_blocks          5329 non-null   float64
 7   a_active_epochs         5388 non-null   int64  
 8   a_attested_epochs       5388 non-null   int64  
 9   a_attested_source       5388 non-null   int64  
 10  a_attested_target       5388 non-null   float64
 11  a_attested_head         5388 non-null   int64  
 12  inc_distance            5388 non-null   float64
 13  sync_rate               5165 non-null   float64
 14  block_reward            5388 non-null   

In [20]:
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X.select_dtypes(include= ['object'])))
# One-hot encoding removed index; put it back
OH_cols_train.index = X.index

In [21]:
OH_cols_train.head()

,0,1,2,3,4,5,6,7,8,9,...,170,171,172,173,174,175,176,177,178,179
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X.select_dtypes(include= ['int64','float64'])
#  Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)

In [28]:
OH_X_train.head()

,total_validators,p_active_slots,p_created_blocks,non_empty_block,empty_block,vanilla_blocks,a_active_epochs,a_attested_epochs,a_attested_source,a_attested_target,...,170,171,172,173,174,175,176,177,178,179
0,10001,83,83,82.0,0.0,51.0,2250225,2249558,2248718,2247582.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,9428,88,88,88.0,0.0,0.0,2121300,2117898,2117410,2116363.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8853,77,77,77.0,0.0,0.0,1991925,1988552,1987913,1989912.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10001,80,80,80.0,0.0,0.0,2250225,2249817,2249157,2248315.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10001,69,69,69.0,0.0,8.0,2250225,2250199,2249164,2247984.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
y.head()

0    0.977869
1    0.968877
2    0.969973
3    0.984954
4    0.985743
Name: effectiveness, dtype: float64

In [24]:
# Splitting 
OH_X_train= OH_X_train.rename(str,axis="columns") 
OH_X_train = OH_X_train.fillna(-1)
train_X, test_X, train_y, test_y = train_test_split(OH_X_train, y, 
					test_size = 0.3, random_state = 123) 

In [31]:
# Instantiation 
model = LinearRegression() 

In [32]:
# Fitting the model 
model.fit(train_X, train_y) 

LinearRegression()

In [33]:
# Predict the model 
pred = model.predict(test_X) 

# RMSE Computation 
rmse = np.sqrt(MSE(test_y, pred)) 
print("RMSE : % f" %(rmse)) 

RMSE :  0.001447


In [34]:
test_y.head()

1266    0.983998
3167    0.981019
1983    0.978622
4835    0.981803
1972    0.970051
Name: effectiveness, dtype: float64

In [35]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# Define the initial types for the ONNX model
initial_type = [('float_input', FloatTensorType([None, OH_X_train.shape[1]]))]

# Convert the scikit-learn model to ONNX
onnx_model = convert_sklearn(model, initial_types=initial_type)

# Save the ONNX model to a file
with open("lr_giza.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [42]:
!giza transpile lr_giza.onnx --output-path verifiable_lr

[giza][2024-06-10 08:18:11.492] No model id provided, checking if model exists ✅
[giza][2024-06-10 08:18:11.494] Model name is: lr_giza
[giza][2024-06-10 08:18:12.273] Model Created with id -> 780! ✅
[giza][2024-06-10 08:18:13.401] Version Created with id -> 1! ✅
[giza][2024-06-10 08:18:13.403] Sending model for transpilation ✅ 
[giza][2024-06-10 08:18:47.333] Transpilation is fully compatible. Version compiled and Sierra is saved at Giza ✅
⠧ Transpiling Model...
[giza][2024-06-10 08:18:48.978] Downloading model ✅
[giza][2024-06-10 08:18:48.985] model saved at: verifiable_lr


In [37]:
test_X.head()

,total_validators,p_active_slots,p_created_blocks,non_empty_block,empty_block,vanilla_blocks,a_active_epochs,a_attested_epochs,a_attested_source,a_attested_target,...,170,171,172,173,174,175,176,177,178,179
1266,8906,61,61,61.0,0.0,0.0,2003850,2003780,2002582,2003042.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3167,8852,54,54,54.0,0.0,0.0,1991505,1991329,1989777,1989013.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1983,8991,94,93,93.0,0.0,9.0,2020630,2020447,2018631,2016855.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4835,8697,85,85,85.0,0.0,56.0,1956825,1956525,1955889,1953060.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1972,500,6,6,6.0,0.0,0.0,112500,112359,112331,112007.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
!giza endpoints deploy --model-id 780 --version-id 1

▰▰▰▰▰▰▱ Creating endpoint!t!
[giza][2024-06-10 08:19:12.870] Endpoint is successful ✅
[giza][2024-06-10 08:19:12.874] Endpoint created with id -> 346 ✅
[giza][2024-06-10 08:19:12.875] Endpoint created with endpoint URL: https://endpoint-ashq-780-1-87735a5d-7i3yxzspbq-ew.a.run.app 🎉


In [48]:
test_X.iloc[0]

total_validators    8906.0
p_active_slots        61.0
p_created_blocks      61.0
non_empty_block       61.0
empty_block            0.0
                     ...  
175                    0.0
176                    0.0
177                    0.0
178                    0.0
179                    0.0
Name: 1266, Length: 200, dtype: float64

In [52]:
from giza.agents.model import GizaModel

MODEL_ID = 780  # Update with your model ID
VERSION_ID = 1  # Update with your version ID

def prediction(input, model_id, version_id):
    model = GizaModel(id=model_id, version=version_id)

    (result, proof_id) = model.predict(
        input_feed={'input': input}, verifiable=True
    ) 

    return result, proof_id

def execution():
    # The input data type should match the model's expected input
    input = test_X.iloc[0].astype(np.float32).values
    (result, proof_id) = prediction(input, MODEL_ID, VERSION_ID)
    print(
        f"Predicted value for input {input.flatten()[0]} is {result[0].flatten()[0]}")
    return result, proof_id


execution()

Payload:  Tensor<FP16x16>
['[200] [583663616 0 3997696 0 3997696 0 3997696 0 0 0 0 0 131324313600 0 131319726080 0 131241213952 0 131271360512 0 129342439424 0 66894 0 6510841 0 585907 0 1506798 0 2092705 0 65536 0 65183 0 65533 0 63858 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 65536 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 65536 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]']


An error occurred in predict: 503 Server Error: Service Unavailable for url: https://endpoint-ashq-780-1-87735a5d-7i3yxzspbq-ew.a.run.app/cairo_run
Deployment predict error: Service Unavailable
An error occurred in predict: 503 Server Error: Service Unavailable for url: https://endpoint-ashq-780-1-87735a5d-7i3yxzspbq-ew.a.run.app/cairo_run


HTTPError: 503 Server Error: Service Unavailable for url: https://endpoint-ashq-780-1-87735a5d-7i3yxzspbq-ew.a.run.app/cairo_run

In [53]:
!giza endpoints logs -e {346} 

[giza][2024-06-10 11:28:18.076] Getting logs for endpoint 346 ✅ 
2024-06-10T02:49:04.953116Z  INFO orion_runner: ✅ Sierra program downloaded successfully!
2024-06-10T02:49:04.953183Z  INFO orion_runner: 🚀 Server running on 0.0.0.0:8080
Default STARTUP TCP probe succeeded after 1 attempt for container "orion-runner-1" on port 8080.
The request failed because either the HTTP response was malformed or connection to the instance had an error. Additional troubleshooting documentation can be found at: https://cloud.google.com/run/docs/troubleshooting#malformed-response-or-connection-error
thread 'tokio-runtime-worker' panicked at crates/cairo_runner/src/utils.rs:16:78:
called `Result::unwrap()` on an `Err` value: FromStrError
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace
2024-06-10T02:53:59.179172Z  INFO orion_runner: ✅ Sierra program downloaded successfully!
2024-06-10T02:53:59.179251Z  INFO orion_runner: 🚀 Server running on 0.0.0.0:8080
Default STARTUP TCP p